<h1/>Imports</h1>

In [1]:
import torch
import numpy as np
from segmentation_model_0001_Preprocess import BUSIDataProcessor, Test_Processor, visualize
from torch.utils.data import DataLoader
from tqdm import tqdm
import random 
import numpy as np
import cv2
import matplotlib.pyplot as plt

C:\Users\maria\anaconda3\envs\tfg\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\maria\anaconda3\envs\tfg\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


<h1>Load data</h1>

In [2]:
DATA_DIR = './data_75_13_12'
         
Experiment_name = 'Prova_submission'
Experiment_folder = os.makedirs(os.path.join('./',Experiment_name), exist_ok = True)
            
x_train_dir = os.path.join(DATA_DIR, 'train')
y_train_dir = os.path.join(DATA_DIR, 'trainannot')
    
x_valid_dir = os.path.join(DATA_DIR, 'val')
y_valid_dir = os.path.join(DATA_DIR, 'valannot')
    
x_test_dir = os.path.join(DATA_DIR, 'test')
y_test_dir = os.path.join(DATA_DIR, 'testannot')

<h1>Load model</h1>

In [3]:
from torch.utils.tensorboard import SummaryWriter
import segmentation_models_pytorch as smp
import torchvision
from torchsummary import summary

ENCODER         = 'resnet18'
ENCODER_WEIGHTS = 'imagenet'
CLASSES         = ['tumor']
ACTIVATION      = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE          = 'cuda'
# ATTENTION       = 'None'
    
# create segmentation model with pretrained encoder
model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    in_channels=3,
    decoder_use_batchnorm=True,
    # decoder_attention_type=ATTENTION,
     activation=ACTIVATION,
    )

In [4]:
model

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

<h1>Train and valid loader</h1>

In [4]:
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS) 

train_dataset = BUSIDataProcessor(
        x_train_dir, 
        y_train_dir,
    )

valid_dataset = Test_Processor(
        x_valid_dir, 
        y_valid_dir,
    )
    
    
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_dataset, batch_size=2, shuffle=True, num_workers=0)


<h1>Loss function-optimizer</h1>

In [5]:
loss = smp.utils.losses.DiceLoss()
    
metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
    ]
    
optimizer = torch.optim.Adam([ 
        dict(params=model.parameters(), lr=0.0001,weight_decay=1e-04),
    ])
    
    
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )
    
valid_epoch = smp.utils.train.ValidEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )


<h1>Train the model</h1>

In [7]:
max_score = 0
min_loss = 10^9
    
writer = SummaryWriter()
num_epochs_decay = 10
    
for i in range(0, 50):
        
        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)
        
        # ...log the running loss
        writer.add_scalars('training and validation loss',
                        {'Train': train_logs[loss.__name__],
                         'Val':valid_logs[loss.__name__]},
                        i)

    
        # Save model
        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            torch.save(model, os.path.join('./',Experiment_name,'segmentation_model_0001.pth'))
            print('Model saved!')
            
        # Decay learning rate                    
        if i + 1 == num_epochs_decay*1:
            optimizer.param_groups[0]['lr'] = 0.00005
            print ('Decay learning rate to lr: {}.'.format(optimizer.param_groups[0]['lr']))
        elif i + 1 == num_epochs_decay*2:
            optimizer.param_groups[0]['lr'] = 0.00001
            print ('Decay learning rate to lr: {}.'.format(optimizer.param_groups[0]['lr']))
        elif i + 1 == num_epochs_decay*3:
            optimizer.param_groups[0]['lr'] = 0.000001
            print ('Decay learning rate to lr: {}.'.format(optimizer.param_groups[0]['lr']))                
        


Epoch: 0
valid: 100%|███████████████████████████████████| 51/51 [00:02<00:00, 25.42it/s, dice_loss - 0.6664, iou_score - 0.3551]
Model saved!

Epoch: 1
valid: 100%|███████████████████████████████████| 51/51 [00:01<00:00, 25.64it/s, dice_loss - 0.5153, iou_score - 0.4933]
Model saved!

Epoch: 2
valid: 100%|███████████████████████████████████| 51/51 [00:02<00:00, 23.65it/s, dice_loss - 0.5267, iou_score - 0.4721]

Epoch: 3
valid: 100%|█████████████████████████████████████| 51/51 [00:02<00:00, 22.61it/s, dice_loss - 0.3863, iou_score - 0.52]
Model saved!

Epoch: 4
valid: 100%|███████████████████████████████████| 51/51 [00:02<00:00, 21.93it/s, dice_loss - 0.2822, iou_score - 0.6331]
Model saved!

Epoch: 5
valid: 100%|█████████████████████████████████████| 51/51 [00:02<00:00, 22.57it/s, dice_loss - 0.2744, iou_score - 0.64]
Model saved!

Epoch: 6
valid: 100%|████████████████████████████████████| 51/51 [00:02<00:00, 22.76it/s, dice_loss - 0.326, iou_score - 0.5893]

Epoch: 7
valid: 100%|███